In [1]:
from string import digits
import re
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk import pos_tag, word_tokenize
from pymorphy2 import MorphAnalyzer
from wordcloud import WordCloud
from matplotlib import pyplot as plt


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 5000)
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_ru')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\astol\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\astol\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\astol\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_ru to
[nltk_data]     C:\Users\astol\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_ru is already up-to-
[nltk_data]       date!


True

In [16]:
tokenizer = nltk.RegexpTokenizer(r"\w+")
morph = MorphAnalyzer()
stopWordsALL = []
stopWordsRU = stopwords.words("russian")
stopWordsEN = stopwords.words("english")
stopWordsALL.extend(stopWordsEN)
punctuations = '''!#$%&'()*,./:;<=>?@\^_`{|}~«»—+₽™⠀"'''

def removeEmoji(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

def lemmatisation(text):
    """Очищает текст от "мусора" (пунктуация, эмодзм)
    Каждое слово ставится в свою первоначальную форму.
    Вернут список строк"""
    tokens = nltk.word_tokenize(removeEmoji(text.lower().translate(str.maketrans('', '', digits))))
    tokens = [morph.parse(token)[0].normal_form for token in tokens if token not in stopWordsALL\
              and token != " " \
              and token.strip() not in punctuations\
              and len(token) > 2]
    return tokens

In [17]:
data = pd.read_json("../data/summary_of_companies.json", orient='index')
companyNames = data.index
data = data.set_index([pd.Index([index for index in range(1, data.shape[0] + 1)])])
data.insert(0, 'Company', companyNames)

In [18]:
# конвентировать список строк в строку.
# None если пустой спикок (компания не казала какую-то конкретную информацию)
for id, company in enumerate(data['Industries'], start=1):
    data['Industries'][id] = ' '.join(data['Industries'][id]) if len(data['Industries'][id]) !=0 else 'None'
    data['Hubs'][id] = ' '.join(data['Hubs'][id]) if len(data['Hubs'][id]) !=0 else 'None'
    data['About'][id] = data['About'][id] if len(data['About'][id]) !=0 else 'None'
    data['Description'][id] = data['Description'][id] if len(data['Description'][id]) !=0 else 'None'
data

C:\Users\astol\Desktop\mo1\module A\Data\ML-Olimp\venv\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\astol\Desktop\mo1\module A\Data\ML-Olimp\venv\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\astol\Desktop\mo1\module A\Data\ML-Olimp\venv\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

,Company,Description,About,Industries,Rating,Subscribers,Hubs,Profile
1,RUVDS.com,VDS/VPS-хостинг. Скидка 15% по коду HABR15,"Международный облачный провайдер RUVDS предлагает в аренду виртуальные серверы, или VPS/VDS. Вам не придется покупать и хранить железо, а также обслуживать и следить за доступностью сервера. Мы делаем это за вас в режиме 24/7. Арендовать серверные мощности и отказаться от них можно в один клик в любое удобное время.\n\nВ 2016 году мы начинали с собственного ЦОДа в подмосковном Королеве и сотрудничества с компанией Huawei, ведущим производителем облачных решений. Сегодня у нас одиннадцать дата-центров: три в Москве, по одному в Питере, Казани, Екатеринбурге, Лондоне, Новосибирске, Цюрихе, Франкфурте и Амстердаме. Распределенная сеть снижает риски клиентов и позволяет арендовать мощности, которые не слишком удалены от офиса или места работы: меньше расстояние — выше скорость.\n\nНаша цель — сделать доступными облачные услуги высокого качества. И кажется, мы хорошо справляемся: среди клиентов — крупный бизнес («ФИНАМ», ВЦИОМ, Gett), независимые ИТ-профессионалы и госкомпании, которым предлагаем наш собственный ЦОД в городе Королеве, сертифицированный ФСТЭК.\n\nВсего за два года нам удалось войти в топ-20 российских поставщиков IAAS-услуг (Infrastructure as a Service — инфраструктура как услуга). Но мы не останавливаемся на достигнутом.\nСейчас команда RUVDS активно работает над проектом раздачи интернета из субкосмоса, развивает форум «Облачные технологии» и полезный блог для программистов, а в свободное время проводит турниры по легендарным видеоиграм 90-х GAME OVERNIGHT, делает вкусные хабрабургеры и даже варит пиво для айтишников.",Связь и телекоммуникации Домены и хостинг Веб-сервисы,2391.52,72000,DIY или Сделай сам Читальный зал Научно-популярное Электроника для начинающих Системное администрирование,https://habr.com/ru/company/ruvds/profile/
2,Selectel,IT-инфраструктура для бизнеса,"Selectel — ведущий в России провайдер облачной инфраструктуры и услуг дата-центров.\n\nМы предлагаем комплексные решения для разных типов организаций — от индивидуальных предпринимателей до крупных корпораций и государственных учреждений. Продукты Selectel позволяют отказаться от капитальных затрат на собственную IT-инфраструктуру, обеспечить ее быструю масштабируемость и высокую производительность.\n\nМы управляем сетью из шести собственных высокотехнологичных дата-центров в Москве, Санкт-Петербурге и Ленинградской области, а также предоставляем сервисы на базе партнерских дата-центров в Новосибирске и Ташкенте.\n\nКаталог продуктов Selectel — один из самых диверсифицированных на рынке. В нем сочетаются продукты для разных уровней IT-инфраструктуры, что позволяет закрыть даже самые нестандартные потребности клиентов.\n\nВыделенные серверы\n\nБольшой выбор готовых конфигураций: от бюджетных вариантов до сложных инфраструктурных проектов. Выделенные серверы обладают «облачной» скоростью развертывания — от 120 секунд. Также есть возможность подобрать конфигурацию, подходящую под индивидуальные задачи, за счет собственной сборки и запасов комплектующих.\n\nОблачная платформа Selectel\n\nВозможность быстрого развертывания облачной инфраструктуры, масштабируемой под задачи различных проектов — от сайтов до сложных распределенных приложений. Selectel самостоятельно разрабатывает и поддерживает ПО платформы на базе Open Source.\n\nПрочие сервисы для комплексного решения задач клиента\n\nОрганизация сети, перенос данных, информационная безопасность, соответствие 152-ФЗ, платформенные сервисы и другие услуги, дополняющие инфраструктуру клиента.\n\nДата-центры Selectel надежны: мы обеспечиваем стабильные каналы связи, бесперебойное питание, охлаждение, круглосуточную охрану и систему мониторинга.\n\nПозиция на рынке: клиенты, рейтинги, награды\n\nМы на рынке с 2008 года, на сегодняшний день у нас более 23 000 клиентов. Нам доверяют такие компании, как X5 Group, Самокат, ПИК-Проект, Учи.ру, YCLIENTS, igooods, amoCRM и другие.\n\nВ 2

In [19]:
# применить лемматизацию
data['Industries'] = data['Industries'].apply(lemmatisation)
data['Hubs'] = data['Hubs'].apply(lemmatisation)
data['Description'] = data['Description'].apply(lemmatisation)
data['About'] = data['About'].apply(lemmatisation)
data

,Company,Description,About,Industries,Rating,Subscribers,Hubs,Profile
1,RUVDS.com,"[vds/vps-хостинг, скидка, код, habr]","[международный, облачный, провайдер, ruvds, предлагать, аренда, виртуальный, сервер, или, vps/vds, вы, прийтись, покупать, хранить, железо, также, обслуживать, следить, доступность, сервер, делать, это, вы, режим, арендовать, серверный, мощность, отказаться, они, можно, один, клик, любой, удобный, время, год, начинать, собственный, цода, подмосковный, королева, сотрудничество, компания, huawei, ведущий, производитель, облачный, решение, сегодня, мы, одиннадцать, дата-центр, три, москва, один, питер, казань, екатеринбург, лондон, новосибирск, цюрих, франкфурт, амстердам, распределённый, сеть, снижать, риск, клиент, позволять, арендовать, мощность, который, слишком, удалить, офис, или, место, работа, маленький, расстояние, выше, скорость, наш, цель, сделать, доступный, облачный, услуга, высокий, качество, казаться, хорошо, справляться, среди, клиент, крупный, бизнес, фина, вциома, gett, ...]","[связь, телекоммуникация, домен, хостинг, веб-сервис]",2391.52,72000,"[diy, или, сделать, сам, читальный, зал, научно-популярный, электроника, для, начинающий, системный, администрирование]",https://habr.com/ru/company/ruvds/profile/
2,Selectel,"[it-инфраструктура, для, бизнес]","[selectel, ведущий, россия, провайдер, облачный, инфраструктура, услуга, дата-центр, предлагать, комплексный, решение, для, разный, тип, организация, индивидуальный, предприниматель, крупный, корпорация, государственный, учреждение, продукт, selectel, позволять, отказаться, капитальный, затрата, собственный, it-инфраструктура, обеспечить, быстрый, масштабируемость, высокий, производительность, управляемый, сеть, шесть, собственный, высокотехнологичный, дата-центр, москва, санкт-петербург, ленинградский, область, также, предоставлять, сервис, база, партнёрский, дата-центр, новосибирск, ташкент, каталог, продукт, selectel, один, самый, диверсифицировать, рынок, немой, сочетаться, продукт, для, разный, уровень, it-инфраструктура, что, позволять, закрыть, даже, самый, нестандартный, потребность, клиент, выделить, сервер, большой, выбор, готовый, конфигурация, бюджетный, вариант, сложный, инфраструктурный, проект, выделить, сервер, обладать, облачный, скорость, развёртывание, секунда, также, есть, возможность, подобрать, конфигурация, подходящий, под, индивидуальный, ...]","[аппаратный, обеспечение, связь, телекоммуникация, домен, хостинг]",1976.50,40000,"[компьютерный, железо, гаджет, it-инфраструктура, it-компания, производство, разработка, электроника]",https://habr.com/ru/company/selectel/profile/
3,Timeweb Cloud,"[облачный, платформа, для, разработчик, бизнес]","[timeweb, cloud, облачный, сервис, сочетать, масса-маркета, enterprise, подход, предоставлять, услуга, виртуальный, машина, россия, европа, аренда, физический, сервер, любой, конфигурация, управляемый, база, данные, решение, контейнеризация, размещать, оборудование, дата-центр, уровень, tier, iii, отвечать, весь, требование, безопасность, соответствовать, фз-, использовать, самый, высокоскоростной, линия, связь, помогать, перенос, настройка, проект, запускать, сложный, инфраструктурный, проект, учёт, требование, клиент, предоставлять, качественный, сервис, опыт, более, год, рынок, подключаться, timeweb, cloud, получать, современный, услуга, dbaas, обеспечивать, полностью, автоматизированный, гибкий, масштабировать, платформа, для, работа, база, данные, объектный, s-хранилище, универсальный, объектный, хранилище, совместимый, amazon, api, надёжный, хранение, мгновенный, доступ, данные, kubernetes, автоматизация, управление, контейнерный, приложение, развёртывание, масштабирование, мониторинг, сервис, балансировщик, нагрузка, ...]","[веб-разработка, домен, хостинг, веб-сервис]",1499.45,1100,"[читальный, зал, научно-популярный, javascript, diy, или, сделать, сам, история]",https://habr.com/ru/company/timeweb/profile/
4,OTUS,"[цифровой, навык, ведущий, эксперт]","[otus, сообщество, професси

In [22]:
def getWordsFromListOfTaggedWords(taggedWords):
    """Отобрать слова от тегов"""
    return [taggedWord[0] for taggedWord in taggedWords]

def getOnlyNeedTags(text):
    """Отобрать только нужные теги"""
    TAGS = ['NONLEX', 'A=m', 'S']
    words = []
    text = word_tokenize(" ".join(text))
    for word, tag in pos_tag(text, lang='rus'):
        if tag in TAGS:
            words.append((word, tag))
    return words

def buildWordCloudComparisonChart(companyName, text, save=False):
    """Построить график "облако слов"
    Облака слов или, как мы привыкли, word clouds — это изображения, которые формируются из списка слов. Обычно, чем чаще слово появляется в списке, тем большего размера или чаще оно будет встречаться в облаке слов. Word cloud необязательно имеет форму облака."""
    companyName = companyName.translate(str.maketrans('', '', punctuations))
    textString = " ".join(text)
    wordCloudBeforeTagging = WordCloud(stopwords=stopWordsALL, width=800, height=800).generate(textString)
    textTaggedString = " ".join(getWordsFromListOfTaggedWords(getOnlyNeedTags(text)))
    wordCloudAfterTagging = WordCloud(stopwords=stopWordsALL, width=800, height=800).generate(textTaggedString)
    fig, (ax1, ax2) = plt.subplots(1, 2, layout='constrained', sharey=True, figsize=(16, 9))
    ax1.imshow(wordCloudBeforeTagging)
    ax2.imshow(wordCloudAfterTagging)
    ax1.axis("off")
    ax2.axis("off")
    fig.suptitle("Отбор тегов", fontsize=20)
    ax1.set_title("До отбора тегов", fontsize=16)
    ax2.set_title("После отбора тегов", fontsize=16)
    if save:
        fig.savefig(f"../data/WordClouds/{companyName}.png")
    else:
        plt.show()
    plt.close(fig)

In [23]:
# графики находятся в data/WordClouds
# Ячейка не выполнена, т.к. графики уже созданы. Нет смысла повторять дорогостоящую операцию.
for companyID in range(1, len(data) + 1):
    buildWordCloudComparisonChart(data["Company"][companyID], data["About"][companyID], save=True)

RUVDScom
